<a href="https://colab.research.google.com/github/ragunggg/routing/blob/main/Web%20App%20Pemod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Package

In [ ]:
!pip install flask-ngrok
!pip install cache-magic
!pip install osmnx
!pip install matplotlib==3.1.3

  Created wheel for cache-magic: filename=cache_magic-1.0.4-cp37-none-any.whl size=6698 sha256=42826e0f49915b8eb6e898b1cf55d979c59470fdf56eadc8887ec72819658b40
  Stored in directory: /root/.cache/pip/wheels/84/3a/44/00b6aea43fe9fcd0c86bbcf33b7e45d167a6b6a1803983325e
Successfully built cache-magic
     |████████████████████████████████| 92kB 5.6MB/s 
     |████████████████████████████████| 1.0MB 19.4MB/s 
     |████████████████████████████████| 6.5MB 44.7MB/s 
     |████████████████████████████████| 61kB 5.7MB/s 
     |████████████████████████████████| 10.3MB 46.2MB/s 
     |████████████████████████████████| 1.0MB 37.8MB/s 
     |████████████████████████████████| 15.3MB 268kB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug

     |████████████████████████████████| 13.1MB 12.2MB/s 
ERROR: osmnx 1.1.0 has requirement matplotlib>=3.3, but you'll have matplotlib 3.1.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.4.1
    Uninstalling matplotlib-3.4.1:
      Successfully uninstalled matplotlib-3.4.1


In [ ]:
import folium
from folium import plugins
import osmnx as ox
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
from branca.element import MacroElement, Figure
import cache_magic
from jinja2 import Template, Environment, BaseLoader
import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
from datetime import datetime
from shapely.geometry import *
from sklearn import model_selection
from sklearn.tree import DecisionTreeRegressor

# Unpickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/pemod'

!ln -s '/content/drive/My Drive/pemod/.cache' /content/.cache

Mounted at /content/drive
/content/drive/My Drive/pemod


In [ ]:
%cache G
%cache betweenness
%cache buildings_weight
%cache -v 35 tseries
#%cache -v 1 model_rf_a
#%cache -v 1 model_rf_b

Loading cached value for variable 'G'. Time since caching: 41 days, 7:19:44.997965
Loading cached value for variable 'betweenness'. Time since caching: 33 days, 23:41:30.142285
Loading cached value for variable 'buildings_weight'. Time since caching: 19 days, 1:18:39.860038
Loading cached value for variable 'tseries'. Time since caching: 7 days, 19:54:12.803718


In [ ]:
length_dict = {}
for u,v,data in G.edges(data=True):
  length_dict[(u,v)] = data.get('length')

In [ ]:
bet_max = max(betweenness.values())
buil_max = max(buildings_weight.values())
l_max = max(length_dict.values())

# Forecasting

In [ ]:
tseries = np.array(tseries)
train = pd.DataFrame({'Date': tseries[:, 0], 'a': tseries[:, 1], 'b': tseries[:, 2]})

In [ ]:
train['Date'] = pd.to_datetime(train['Date'])
train['hour'] = pd.DatetimeIndex(train['Date']).hour
train['minute'] = pd.DatetimeIndex(train['Date']).minute
train['second'] = pd.DatetimeIndex(train['Date']).second
train['weekday'] = pd.DatetimeIndex(train['Date']).weekday
train = pd.get_dummies(train, columns=['weekday'], drop_first=True, prefix='wday')

In [ ]:
target_column_a = ['a'] 
target_column_b = ['b']
predictors = ['hour',  'minute',  'second',  'wday_1',  'wday_2', 'wday_3', 'wday_4', 'wday_5', 'wday_6']

In [ ]:
x_train = train[predictors].values
y_train_a = train[target_column_a].values.astype(float)
y_train_b = train[target_column_b].values.astype(float)

In [ ]:
dtree1 = DecisionTreeRegressor(max_depth=7)
dtree2 = DecisionTreeRegressor(max_depth=7)
dtree1.fit(x_train, y_train_a)
dtree2.fit(x_train, y_train_b)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=7,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

In [ ]:
def bobot_G(func):
  def inner(G):
    H = G
    a, b = func()

    for u,v,data in H.edges(data=True):
      try:
        data['bobot'] = (a*(betweenness[(u,v)]/bet_max)+b*(buildings_weight[(u,v)]/buil_max))/(data['length']/l_max)
      except:
        data['bobot'] = (a*(betweenness[(u,v)]/bet_max))/(data['length']/l_max)

    return H

  return inner

In [ ]:
@bobot_G
def prediction():
  dt = datetime.today()
  h = dt.hour
  m = dt.minute
  s = dt.second
  w = dt.weekday()
  wday_1, wday_2, wday_3, wday_4, wday_5, wday_6 = 0,0,0,0,0,0

  if w==1:
    wday_1 = 1
  elif w==2:
    wday_2 = 1
  elif w==3:
    wday_3 = 1
  elif w==4:
    wday_4 = 1
  elif w==5:
    wday_5 = 1
  elif w==6:
    wday_6 = 1

  features = np.array([h,  m,  s, wday_1,  wday_2, wday_3, wday_4, wday_5, wday_6]).reshape(1,-1)
  pred_a = float(dtree1.predict(features))
  pred_b = float(dtree2.predict(features))

  return pred_a, pred_b

# Flask Setup

In [ ]:
north, east, south, west = -6.8686, 107.636, -6.9181, 107.5862

In [ ]:
coordinates = [[north,west],
               [north,east],
               [south,east],
               [south,west],
               [north,west]]

In [ ]:
bounding_PolyLine = folium.PolyLine(locations=coordinates,weight=5)
tooltip = "Tekan!"
weights = [i for i in range(1,6)]
distances = [i for i in range(1000,5001,1000)]

In [ ]:
layout = Environment(loader=BaseLoader()).from_string("""
<!doctype html>
<html lang="en">
  <head>
    <!-- Required meta tags -->
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width, height=device-height, initial-scale=1, shrink-to-fit=no">

    <!-- Bootstrap CSS -->
    <link href="//maxcdn.bootstrapcdn.com/bootstrap/3.3.0/css/bootstrap.min.css" rel="stylesheet" id="bootstrap-css">
    <script type="text/javascript" src="//ajax.googleapis.com/ajax/libs/jquery/1.9.1/jquery.min.js"></script>
    <script type="text/javascript" src="//maxcdn.bootstrapcdn.com/bootstrap/3.3.0/js/bootstrap.min.js"></script>
    <script type="text/javascript" src="//code.jquery.com/jquery-1.11.1.min.js"></script>

    <style type="text/css">
    html, body {
      height : 100vh;
    }
    </style>

    <title>Pemodelan Matematika</title>
  </head>

  <body>
    <nav class="navbar navbar-default navbar-fixed-top">
      <div class="container-fluid">
        <!-- Brand and toggle get grouped for better mobile display -->
        <div class="navbar-header">
          <button type="button" class="navbar-toggle collapsed" data-toggle="collapse" data-target="#bs-example-navbar-collapse-1" aria-expanded="false">
            <span class="sr-only">Toggle navigation</span>
            <span class="icon-bar"></span>
            <span class="icon-bar"></span>
            <span class="icon-bar"></span>
          </button>
          <a class="navbar-brand" href="/">Pemod</p></a>
        </div>

        <!-- Collect the nav links, forms, and other content for toggling -->
        <div class="collapse navbar-collapse" id="bs-example-navbar-collapse-1">
          <ul class="nav navbar-nav">
            <li><a href="/home">Home<span class="sr-only">(current)</span></a></li>
            <li><a href="/isoweights">Isoweights</a></li>
            <li><a href="/isodistances">Isodistances</a></li>
          </ul>
        </div><!-- /.navbar-collapse -->
      </div><!-- /.container-fluid -->
    </nav>
    
    {% block body %}{% endblock %}
  </body>
</html>
""")

In [ ]:
def html_template(m):
  folium.TileLayer('Stamen Terrain').add_to(m)
  folium.TileLayer('Stamen Toner').add_to(m)
  folium.TileLayer('Stamen Water Color').add_to(m)
  folium.TileLayer('cartodbpositron').add_to(m)
  folium.TileLayer('cartodbdark_matter').add_to(m)
  folium.LayerControl().add_to(m)

  m.add_child(bounding_PolyLine)

  fig = Figure(height="100%")

  fig.add_child(m)
  
  map_content = Environment(loader=BaseLoader()).from_string(m._repr_html_())

  index = Environment(loader=BaseLoader()).from_string("""
  {% extends layout %}
  {% block body %}
    {% include map_content %}
  {% endblock %}
  """)

  return index.render(layout=layout,map_content=map_content)

In [ ]:
def get_isoweights(G,y,x):
  H = prediction(G)
  center_node = ox.distance.nearest_nodes(H, x, y)
  
  isoweights_polys = []
  for weight in weights:
    subgraph = nx.ego_graph(H, center_node, radius=weight, distance='bobot')
    node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
    bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
    isoweights_polys.append(bounding_poly)

  return isoweights_polys

In [ ]:
def get_isodistances(G,y,x):
  H = prediction(G)
  center_node = ox.distance.nearest_nodes(H, x, y)
  
  isodistances_polys = []
  for distance in sorted(distances):
    subgraph = nx.ego_graph(H, center_node, radius=distance, distance='length')
    node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
    bounding_poly = gpd.GeoSeries(node_points).unary_union.convex_hull
    isodistances_polys.append(bounding_poly)

  return isodistances_polys

In [ ]:
class folium_route(object):
  def __init__(self,G,start,end):
    self.H = prediction(G)
    self.start = start
    self.end = end

  def shortest_route(self):
    start_ID = ox.distance.nearest_nodes(self.H, self.start[1], self.start[0])
    end_ID = ox.distance.nearest_nodes(self.H, self.end[1], self.end[0])
    least_weight = ox.shortest_path(self.H, start_ID, end_ID, weight = 'bobot')
    shortest_distance = ox.shortest_path(self.H, start_ID, end_ID, weight = 'length')
    
    least_weight_length = nx.path_weight(self.H, least_weight, weight="length")/1000
    shortest_distance_length = nx.path_weight(self.H, shortest_distance, weight="length")/1000
    
    return least_weight, shortest_distance, least_weight_length, shortest_distance_length

  def feature_route(self,m,fg1,fg2):
    path1, path2, path_length1, path_length2 = self.shortest_route()
    ox.folium.plot_route_folium(G, path1, route_map=fg1, zoom = 14,tiles='OpenStreetMap',tooltip='<b>Jarak Tempuh : {} km</b>'.format(round(path_length1,2))).add_to(m)
    ox.folium.plot_route_folium(G, path2, route_map=fg2, zoom = 14,tiles='OpenStreetMap',tooltip='<b>Jarak Tempuh : {} km</b>'.format(round(path_length2,2)),color="green").add_to(m)

In [ ]:
class LatLngPopup_init(MacroElement):
    _template = Template(u"""
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.popup();
                function latLngPop(e) {
                    {{this.get_name()}}
                        .setLatLng(e.latlng)
                        .setContent( "<a style=text-decoration:none;color:#28b5b5 onclick=window.open('/{{this.route}}?{{this.params_lat}}="+e.latlng.lat.toFixed(4)+"&{{this.params_lon}}="+ e.latlng.lng.toFixed(4)+"')> <p>Klik Disini</p> </a>")
                        .openOn({{this._parent.get_name()}})
                    }
                {{this._parent.get_name()}}.on('click', latLngPop);

            {% endmacro %}
            """)  # noqa

    def __init__(self,route,params_lat,params_lon):
        super(LatLngPopup_init, self).__init__()
        self._name = 'LatLngPopup_init'
        self.route = route
        self.params_lat = params_lat
        self.params_lon = params_lon

In [ ]:
class LatLngPopup(MacroElement):
    _template = Template(u"""
            {% macro script(this, kwargs) %}
                var {{this.get_name()}} = L.popup();
                function latLngPop(e) {
                    {{this.get_name()}}
                        .setLatLng(e.latlng)
                        .setContent( "<a style=text-decoration:none;color:#28b5b5 onclick=window.open('"+window.location.href+"&{{this.params_lat}}="+e.latlng.lat.toFixed(4)+"&{{this.params_lon}}="+ e.latlng.lng.toFixed(4)+"')> <p>Klik Disini</p> </a>")
                        .openOn({{this._parent.get_name()}})
                    }
                {{this._parent.get_name()}}.on('click', latLngPop);

            {% endmacro %}
            """)  # noqa

    def __init__(self,params_lat,params_lon):
        super(LatLngPopup, self).__init__()
        self._name = 'LatLngPopup'
        self.params_lat = params_lat
        self.params_lon = params_lon

# Flask

In [ ]:
app = Flask(__name__)

run_with_ngrok(app)

@app.route('/')
@app.route('/home')
def home_page():
  m = folium.Map(location=[-6.8906, 107.6108], zoom_start=14, top=50)

  m.add_child(LatLngPopup_init('route','start_lat','start_lon'))

  return html_template(m)

@app.route('/route')
def route():
  start_lat  = float(request.args.get('start_lat', -10000))
  start_lon  = float(request.args.get('start_lon', -10000))
  end_lat  = float(request.args.get('end_lat', -10000))
  end_lon  = float(request.args.get('end_lon', -10000))

  m = folium.Map(location=[-6.8906, 107.6108], zoom_start=14,top=50)

  if (end_lat != -10000) and (end_lon != -10000):
    start = (start_lat,start_lon)
    end = (end_lat,end_lon)

    fg1 = folium.FeatureGroup(name='least weight')

    fg2 = folium.FeatureGroup(name='shortest distance',show=False)

    route = folium_route(G,start,end)

    route.feature_route(m,fg1,fg2)

    folium.Marker([start_lat, start_lon], popup="<b>Lokasi Sekarang</b>", tooltip=tooltip).add_to(m)

    folium.Marker([end_lat, end_lon], popup="<b>Tujuan</b>", tooltip=tooltip).add_to(m)

    m.add_child(LatLngPopup_init('route','start_lat','start_lon'))

    return html_template(m)

  folium.Marker([start_lat, start_lon], popup="<b>Lokasi Sekarang</b>", tooltip=tooltip).add_to(m)

  m.add_child(LatLngPopup('end_lat', 'end_lon'))
  
  return html_template(m)

@app.route('/isoweights')
def isoweights():
  center_lat = float(request.args.get('center_lat', -10000))
  center_lon = float(request.args.get('center_lon', -10000))

  m = folium.Map(location=[-6.8906, 107.6108], zoom_start=14,top=50)

  if (center_lat != -10000) and (center_lon != -10000):

    folium.Marker([center_lat, center_lon], popup="<b>Pusat</b>", tooltip=tooltip).add_to(m)

    isoweights_polys = get_isoweights(G,center_lat, center_lon)

    i = 1
    for polygon in isoweights_polys:
      fg = folium.FeatureGroup(name=f"Bobot {i}").add_to(m)
      folium.GeoJson(polygon).add_to(fg)
      i+=1

    m.add_child(LatLngPopup_init('isoweights','center_lat', 'center_lon'))

    return html_template(m)

  m.add_child(LatLngPopup_init('isoweights','center_lat', 'center_lon'))

  return html_template(m)

@app.route('/isodistances')
def isodistances():
  center_lat = float(request.args.get('center_lat', -10000))
  center_lon = float(request.args.get('center_lon', -10000))

  m = folium.Map(location=[-6.8906, 107.6108], zoom_start=14,top=50)

  if (center_lat != -10000) and (center_lon != -10000):
    folium.Marker([center_lat, center_lon], popup="<b>Pusat</b>", tooltip=tooltip).add_to(m)

    isodistances_polys = get_isodistances(G,center_lat, center_lon)

    i = 1
    for polygon in isodistances_polys:
      fg = folium.FeatureGroup(name=f"Panjang {i} km").add_to(m)
      folium.GeoJson(polygon).add_to(fg)
      i+=1

    m.add_child(LatLngPopup_init('isodistances','center_lat', 'center_lon'))

    return html_template(m)

  m.add_child(LatLngPopup_init('isodistances','center_lat', 'center_lon'))

  return html_template(m)

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://48452bf334f0.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [07/May/2021 06:16:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2021 06:16:47] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/May/2021 06:17:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2021 06:17:03] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/May/2021 06:17:13] "GET / HTTP/1.1" 200 -
